<a href="https://colab.research.google.com/github/kunmukh/HackUTD/blob/master/NLPHomeWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
def process(txt):
    return txt.strip('.').replace(',','').lower()
def bigram(txt):
    x = process(txt).split()
    if len(x) == 1:
        return [(x[0],'')]
    return [(x[i],x[i+1]) for i in range(len(x)-1)]

In [3]:
S1 = "Sales of the company to return to normalcy."
S2 = "The new products and services contributed to increase revenue."

##Trigram without smoothing


In [140]:
sentence = S1

In [141]:
df = pd.DataFrame(0, index=bigram(sentence), columns=set(process(sentence).split()))

In [142]:
f = open("//content/corpus_for_language_models.txt", "r")
corpus = f.read()

In [143]:
corpus = process(corpus)

In [144]:
import nltk
nltk.download('punkt')
ngrams = {}
words = 2
idx = 0
words_tokens = nltk.word_tokenize(corpus)
for i in range(len(words_tokens)-words):
    seq = ' '.join(words_tokens[i:i+words])
    if  seq not in ngrams.keys():
        ngrams[seq] = []
    ngrams[seq].append(words_tokens[i+words])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [145]:
from collections import Counter

for rows in df.index:
  tmp=rows[0]+' '+rows[1]
  if tmp in ngrams:
    d_tmp = Counter(ngrams[tmp])
    for col in df.columns:
      if col in d_tmp:
        df.loc[[rows],[col]]= d_tmp[col]
df
  

,sales,of,return,to,the,company,normalcy
"(sales, of)",0,0,0,0,0,0,0
"(of, the)",0,0,0,0,0,17,0
"(the, company)",0,0,0,2,0,0,0
"(company, to)",0,0,1,0,0,0,0
"(to, return)",0,0,0,1,0,0,0
"(return, to)",0,0,0,0,2,0,1
"(to, normalcy)",0,0,0,0,0,0,0


In [124]:
from collections import defaultdict
d_bigram_count = defaultdict(None)

for rows in df.index:
  tmp=rows[0]+' '+rows[1]
  d_bigram_count[tmp] = len(ngrams[tmp])

d_bigram_count

defaultdict(<function __main__.rer>,
            {'and services': 2,
             'contributed to': 2,
             'increase revenue': 1,
             'new products': 1,
             'products and': 3,
             'services contributed': 1,
             'the new': 36,
             'to increase': 2})

In [121]:
df_prob = df.copy()

for rows in df_prob.index:
  tmp=rows[0]+' '+rows[1]
  if tmp in d_bigram_count:
    df_prob.loc[[rows]] = df_prob.loc[[rows]] / d_bigram_count[tmp]

df_prob

,new,revenue,contributed,products,services,increase,to,the,and
"(the, new)",0.0,0.0,0.0,0.027778,0.000000,0.0,0.0,0.0,0.0
"(new, products)",0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
"(products, and)",0.0,0.0,0.0,0.000000,0.333333,0.0,0.0,0.0,0.0
"(and, services)",0.0,0.0,0.5,0.000000,0.000000,0.0,0.0,0.0,0.0
"(services, contributed)",0.0,0.0,0.0,0.000000,0.000000,0.0,1.0,0.0,0.0
"(contributed, to)",0.0,0.0,0.0,0.000000,0.000000,0.5,0.0,0.5,0.0
"(to, increase)",0.0,0.5,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
"(increase, revenue)",0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0


In [122]:

words = 2
tmpw = ["."] + process(sentence).split()[:2]
P = Counter(ngrams[tmpw[0]+ " " + tmpw[1]])[tmpw[-1]] / len(ngrams[tmpw[0]+ " " + tmpw[1]])
words_tokens = nltk.word_tokenize(process(sentence))
for i in range(len(words_tokens)-words):
    x_search = tuple(words_tokens[i:i+words])
    y_search = words_tokens[i+words]
    #if float(df_prob.loc[[x_search]][y_search]):
    P *= float(df_prob.loc[[x_search]][y_search])

print(P)

0.0


##Trigram with Laplace smoothing


In [146]:
sentence = S1
df = pd.DataFrame(1, index=bigram(sentence), columns=set(process(sentence).split()))
df

,sales,of,return,to,the,company,normalcy
"(sales, of)",1,1,1,1,1,1,1
"(of, the)",1,1,1,1,1,1,1
"(the, company)",1,1,1,1,1,1,1
"(company, to)",1,1,1,1,1,1,1
"(to, return)",1,1,1,1,1,1,1
"(return, to)",1,1,1,1,1,1,1
"(to, normalcy)",1,1,1,1,1,1,1


In [149]:


f = open("//content/corpus_for_language_models.txt", "r")
corpus = f.read()
corpus = process(corpus)
import nltk
nltk.download('punkt')
ngrams = {}
words = 2
idx = 0
words_tokens = nltk.word_tokenize(corpus)
for i in range(len(words_tokens)-words):
    seq = ' '.join(words_tokens[i:i+words])
    if  seq not in ngrams.keys():
        ngrams[seq] = []
    ngrams[seq].append(words_tokens[i+words])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [148]:
from collections import Counter

for rows in df.index:
  tmp=rows[0]+' '+rows[1]
  if tmp in ngrams:
    d_tmp = Counter(ngrams[tmp])
    for col in df.columns:
      if col in d_tmp:
        df.loc[[rows],[col]]+= d_tmp[col]

df

,sales,of,return,to,the,company,normalcy
"(sales, of)",1,1,1,1,1,1,1
"(of, the)",1,1,1,1,1,18,1
"(the, company)",1,1,1,3,1,1,1
"(company, to)",1,1,2,1,1,1,1
"(to, return)",1,1,1,2,1,1,1
"(return, to)",1,1,1,1,3,1,2
"(to, normalcy)",1,1,1,1,1,1,1


In [150]:
from collections import defaultdict
d_bigram_count = defaultdict(None)

for rows in df.index:
  tmp=rows[0]+' '+rows[1]
  d_bigram_count[tmp] = len(ngrams[tmp])

V = len(set(words_tokens))

for keys in d_bigram_count.keys():
  d_bigram_count[keys] += V

d_bigram_count

defaultdict(None,
            {'company to': 5248,
             'of the': 5512,
             'return to': 5249,
             'sales of': 5246,
             'the company': 5337,
             'to normalcy': 5246,
             'to return': 5247})

In [151]:
df_prob = df.copy()

for rows in df_prob.index:
  tmp=rows[0]+' '+rows[1]
  if tmp in d_bigram_count:
    df_prob.loc[[rows]] = df_prob.loc[[rows]] / d_bigram_count[tmp]

df_prob

,sales,of,return,to,the,company,normalcy
"(sales, of)",0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191
"(of, the)",0.000181,0.000181,0.000181,0.000181,0.000181,0.003266,0.000181
"(the, company)",0.000187,0.000187,0.000187,0.000562,0.000187,0.000187,0.000187
"(company, to)",0.000191,0.000191,0.000381,0.000191,0.000191,0.000191,0.000191
"(to, return)",0.000191,0.000191,0.000191,0.000381,0.000191,0.000191,0.000191
"(return, to)",0.000191,0.000191,0.000191,0.000191,0.000572,0.000191,0.000381
"(to, normalcy)",0.000191,0.000191,0.000191,0.000191,0.000191,0.000191,0.000191


In [152]:
words = 2
tmpw = ["."] + process(sentence).split()[:2]
P = (Counter(ngrams[tmpw[0]+ " " + tmpw[1]])[tmpw[-1]] + 1) / (len(ngrams[tmpw[0]+ " " + tmpw[1]]) + V)
words_tokens = nltk.word_tokenize(process(sentence))
for i in range(len(words_tokens)-words):
    x_search = tuple(words_tokens[i:i+words])
    y_search = words_tokens[i+words]
    #if float(df_prob.loc[[x_search]][y_search]):
    P *= float(df_prob.loc[[x_search]][y_search])

print(P)


7.383619276130004e-24


In [154]:
import pandas as pd
import re
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')


#Function using stanford POS
def POS_NLTK():
    sentence1 = 'The chairman of the board is completely bold .'
    print("\nFor sentence 1: ",sentence1)

    #for 1st sentence
    output = pos_tag(sentence1.split(' '))
    print(output)

    sentence2 = 'A chair was found in the middle of the road .'
    print("\nFor sentence 2: ",sentence2)

    #for 1st sentence
    output =pos_tag(sentence2.split(' '))
    print(output)

#declaring transmission matrix
trans_matrix = pd.DataFrame({'tags':['DT','NN','VB','VBZ','VBN','JJ','RB','IN','</s>'],'<s>':[0.38, 0.32, 0.05, 0, 0, 0.11, 0.1, 0.23, 0],'DT':[0, 0.58, 0, 0, 0, 0.42, 0, 0, 0,],'NN':[0, 0.12, 0, 0.05, 0.32, 0, 0, 0.25, 0.11],'VB':[0.01, 0.05, 0, 0, 0, 0, 0.2, 0.61, 0.13],'VBZ':[0.2, 0.3, 0, 0, 0, 0.25, 0.15, 0.1, 0],'VBN':[0.18, 0.22, 0, 0, 0.2, 0.07, 0.16, 0.11, 0.06],'JJ':[0, 0.88, 0, 0, 0, 0.12, 0, 0, 0],'RB':[0, 0, 0, 0.22, 0.28, 0.39, 0.1, 0, 0.01],'IN':[0.57, 0.28, 0, 0, 0, 0.15, 0, 0, 0]})
trans_matrix = trans_matrix.set_index(['tags'])
trans_matrix = trans_matrix.T
trans_matrix.columns = ('DT','NN','VB','VBZ','VBN','JJ','RB','IN','</s>')

#declaring emission matrix
emission_mat = pd.DataFrame({'tags':['a', 'the', 'chair', 'chairman', 'board', 'road', 'is', 'was', 'found', 'middle', 'bold', 'completely', 'in', 'of'],'DT':[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],'NN':[0, 0, 0.69, 1, 0.88, 1, 0, 0, 0.01, 0.66, 0.38, 0, 0, 0],'VB':[0, 0, 0.31, 0, 0.12, 0, 0, 0, 0, 0, 0, 0, 0, 0],'VBZ':[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],'VBN':[0, 0, 0, 0, 0, 0, 0, 1, 0.99, 0, 0, 0, 0, 0],'JJ':[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.34, 0.62, 0, 0, 0],'RB':[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],'IN':[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]})
emission_mat = emission_mat.set_index(['tags'])
emission_mat = emission_mat.T
emission_mat.columns = ('a', 'the', 'chair', 'chairman', 'board', 'road', 'is', 'was', 'found', 'middle', 'bold', 'completely', 'in', 'of')


#declaring viterbi table
viterbi_table=  pd.DataFrame({'tags':['a', 'the', 'chair', 'chairman', 'board', 'road', 'is', 'was', 'found', 'middle', 'bold', 'completely', 'in', 'of'],'DT':[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],'NN':[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],'VB':[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],'VBZ':[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],'VBN':[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],'JJ':[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],'RB':[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],'IN':[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]})
viterbi_table = viterbi_table.set_index(['tags'])
viterbi_table = viterbi_table.T
viterbi_table.columns = ('a', 'the', 'chair', 'chairman', 'board', 'road', 'is', 'was', 'found', 'middle', 'bold', 'completely', 'in', 'of')

#declaring back pointer table
bk_ptr=  pd.DataFrame({'tags':['a', 'the', 'chair', 'chairman', 'board', 'road', 'is', 'was', 'found', 'middle', 'bold', 'completely', 'in', 'of'],'DT':["","","","","","","","","","","","","",""],'NN':["","","","","","","","","","","","","",""],'VB':["","","","","","","","","","","","","",""],'VBZ':["","","","","","","","","","","","","",""],'VBN':["","","","","","","","","","","","","",""],'JJ':["","","","","","","","","","","","","",""],'RB':["","","","","","","","","","","","","",""],'IN':["","","","","","","","","","","","","",""]})
bk_ptr = bk_ptr.set_index(['tags'])
bk_ptr = bk_ptr.T
bk_ptr.columns = ('a', 'the', 'chair', 'chairman', 'board', 'road', 'is', 'was', 'found', 'middle', 'bold', 'completely', 'in', 'of')




#viterbi algorithm
def viterbi_algorithm(tags, sentence, emission_mat, trans_matrix):

    #initiliaze viterbi table
    for s in tags:
        viterbi_table[sentence[0]][s] = trans_matrix[s]['<s>'] * emission_mat[sentence[0]][s]
        bk_ptr[sentence[0]][s] = None
       
    len1=len(sentence)
    len2= len(tags)

    #finding viterbi table's probability using emission matrix and transmission matrix              
    for word in range(1,len1):
        for tag in range(0,len2 ):        
       
            max_probability = viterbi_table[sentence[word-1]][tags[0]] * trans_matrix[tags[tag]][tags[0]] * emission_mat[sentence[word]][tags[tag]]
            current_tag = tags[0]

            for prev_tag in tags[1:]:
                
                current_probability = viterbi_table[sentence[word - 1]][prev_tag] * trans_matrix[tags[tag]][prev_tag] * emission_mat[sentence[word]][tags[tag]]
             
                if current_probability > max_probability:
                    max_probability = current_probability
                    current_tag = prev_tag
                                    
            viterbi_table[sentence[word]][tags[tag]] = max_probability
            bk_ptr[sentence[word]][tags[tag]] = current_tag



sentence1 = "the chairman of the board is completely bold".split()

#viterbi for first sentence
viterbi_algorithm(emission_mat.index, sentence1, emission_mat, trans_matrix)

print("\n******************* Using Viterbi  *******************\n")


print("\n******************* Viterbi Table for sentence 1 *******************\n")

print(viterbi_table)
print("\nThe probability of the tag sequence for sentence 1:", viterbi_table[sentence1[-1]][:].max())
print("\n******************* Viterbi Table for sentence 2 *******************\n")
sentence2 ="a chair was found in the middle of the road".split()
#viterbi for second sentence

viterbi_algorithm(emission_mat.index, sentence2, emission_mat, trans_matrix)
print(viterbi_table)
print("\nThe probability of the tag sequence for sentence 2:", viterbi_table[sentence2[-1]][:].max())


print("\n******************* Using NLTK Stanford POS *******************\n")

#runing Stnafor POS for both sentences
POS_NLTK()












[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.

******************* Using Viterbi  *******************


******************* Viterbi Table for sentence 1 *******************

       a       the  chair  chairman  ...      bold  completely   in      of
DT   0.0  0.031407    0.0    0.0000  ...  0.000000     0.00000  0.0  0.0000
NN   0.0  0.000000    0.0    0.2204  ...  0.000000     0.00000  0.0  0.0000
VB   0.0  0.000000    0.0    0.0000  ...  0.000000     0.00000  0.0  0.0000
VBZ  0.0  0.000000    0.0    0.0000  ...  0.000000     0.00000  0.0  0.0000
VBN  0.0  0.000000    0.0    0.0000  ...  0.000000     0.00000  0.0  0.0000
JJ   0.0  0.000000    0.0    0.0000  ...  0.000029     0.00000  0.0  0.0000
RB   0.0  0.000000    0.0    0.0000  ...  0.000000     0.00012  0.0  0.0000
IN   0.0  0.000000    0.0    0.0000  ...  0.000000     0.00000  0.0  0.0551

[8 rows x 14 columns]

The

In [155]:
trans_matrix

,DT,NN,VB,VBZ,VBN,JJ,RB,IN,</s>
<s>,0.38,0.32,0.05,0.00,0.00,0.11,0.10,0.23,0.00
DT,0.00,0.58,0.00,0.00,0.00,0.42,0.00,0.00,0.00
NN,0.00,0.12,0.00,0.05,0.32,0.00,0.00,0.25,0.11
VB,0.01,0.05,0.00,0.00,0.00,0.00,0.20,0.61,0.13
VBZ,0.20,0.30,0.00,0.00,0.00,0.25,0.15,0.10,0.00
VBN,0.18,0.22,0.00,0.00,0.20,0.07,0.16,0.11,0.06
JJ,0.00,0.88,0.00,0.00,0.00,0.12,0.00,0.00,0.00
RB,0.00,0.00,0.00,0.22,0.28,0.39,0.10,0.00,0.01
IN,0.57,0.28,0.00,0.00,0.00,0.15,0.00,0.00,0.00
